In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd
import os 

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader

from dataset.data_loader_RGB import *
from utils.pytorch_ssim import *

from facenet_pytorch import InceptionResnetV1
from models.GAN_coord_model import Generator, Discriminator
from utils.loss import*

torch.manual_seed(1)
device = torch.device("cuda")

In [2]:
# SRGAN parameters

batch_size = 32
num_epochs = 10
lr = 1e-4
threads = 4
upscale_factor = 4

In [3]:
#img_path_low = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-56/train'
#img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

img_path_low = '/home/jupyter/dataset/LR_56/train/'
img_path_ref = '/home/jupyter/dataset/HR/train/'

#img_path_low = '/media/angelo/DATEN/Datasets/CelebA/LR_56/train/'
#img_path_ref = '/media/angelo/DATEN/Datasets/CelebA/HR/train/'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [4]:
netG = Generator(upscale_factor).to(device)
netD = Discriminator().to(device)

#netG = torch.load('models/SRGAN_coord_FaceLoss_x4_epoch_1.pth', map_location=lambda storage, loc: storage).to(device)
#netD = torch.load('models/netD_FaceLoss_x4_epoch_12.pth', map_location=lambda storage, loc: storage).to(device)

#netG.weight_init(mean=0.0, std=0.2)
#netD.weight_init(mean=0.0, std=0.2)

feature_extraction_model = InceptionResnetV1(pretrained='vggface2').eval()

generator_criterion = GeneratorLoss().to(device)
face_loss = FaceIdentityLoss(feature_extraction_model).to(device)
MSE_criterion = nn.MSELoss().to(device)

In [5]:
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
    
results = {'d_loss': [], 'g_loss': [], 'd_score': [],
           'g_score': [], 'psnr': [], 'ssim': []}

In [ ]:
# Let's pretrain the Generator
pre_train_epochs = 5
for epoch in range(1, pre_train_epochs+1):
    netG.train()
    
    for data, target in training_data_loader:
        data, target = data.to(device), target.to(device)
        netG.zero_grad()
        loss = MSE_criterion(netG(data), target)
        loss.backward()
        optimizerG.step()
    
    print('Last loss: {}'.format(loss.item()))
    print('Finished pre-training {}/{}'.format(epoch,pre_train_epochs))

In [6]:
out_path = 'results/'
out_model_path = 'checkpoints/'

if not os.path.exists(out_path):
    os.makedirs(out_path)    

if not os.path.exists(out_model_path):
    os.makedirs(out_model_path)  

In [7]:
def train(epoch):
    
    running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 
                       'd_score': 0, 'g_score': 0}
    
    netG.train()
    netD.train()
    iteration = 0
    for data, target in training_data_loader:
        iteration+=1
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        ############################
        # (1) Update D network: maximize D(x)-1-D(G(z))
        ###########################
        real_img = Variable(target).to(device)
        z = Variable(data).to(device)
        
        fake_img = netG(z)

        netD.zero_grad()
        
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        
        d_loss = 1 - real_out + fake_out
        
        if d_loss.item() > 0.5:
            d_loss.backward(retain_graph=True)
            optimizerD.step()

        ############################
        # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss + FaceID Loss
        ###########################
        netG.zero_grad()
        
        #mse_adversarial_content_loss = generator_criterion(fake_out, fake_img, real_img)
        #face_perceptual_loss = face_loss(fake_img, real_img)
        
        g_loss = generator_criterion(fake_out, fake_img, real_img) + face_loss(fake_img, real_img)
        
        g_loss.backward()

        fake_img = netG(z)
        fake_out = netD(fake_img).mean()

        optimizerG.step()

        # Loss for current batch before optimization 

        running_results['g_loss'] += g_loss.item() * batch_size
        running_results['d_loss'] += d_loss.item() * batch_size
        running_results['d_score'] += real_out.item() * batch_size
        running_results['g_score'] += fake_out.item() * batch_size

        print("===> Epoch[{}]({}/{}): D(x) {:.4f} Loss_D: {:.4f}".format(epoch, iteration, 
                                                                         len(training_data_loader), 
                                                                         real_out.item(),
                                                                         d_loss.item()))

    batch_mse = ((fake_img - real_img) ** 2).data.mean()
    batch_ssim = ssim(fake_img, real_img).item()
    batch_psnr = 10 * log10(1 /batch_mse)  
    
    print('[{}/{}] Loss_D: {:.4f} Loss_G: {:.4f} D(x): {:.4f} D(G(z)): {:.4f} PSNR {:.4f}'.format(
        epoch, num_epochs, 
        running_results['d_loss'] / running_results['batch_sizes'],
        running_results['g_loss'] / running_results['batch_sizes'],
        running_results['d_score'] / running_results['batch_sizes'],
        running_results['g_score'] / running_results['batch_sizes'], batch_psnr))
          
    # Save loss\scores\psnr\ssim
    results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
    results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
    results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
    results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
    results['psnr'].append(batch_psnr)
    results['ssim'].append(batch_ssim)

    if epoch % 2 == 0:

        # Save model parameters
        torch.save(netG, out_model_path + 'SRGAN_coord_FaceLoss_x%d_epoch_%d.pth' % (upscale_factor, epoch))
        torch.save(netD, out_model_path + 'netD_coord_FaceLoss_x%d_epoch_%d.pth' % (upscale_factor, epoch))

        data_frame = pd.DataFrame(
            data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                  'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
            index=range(1, epoch + 1))
        data_frame.to_csv(out_path + 'SRGAN_coord_FaceLossID_x' + str(upscale_factor) + '_train_results.csv', index_label='Epoch')

In [ ]:
#num_epochs = 30

for epoch in range(11, 31):
    train(epoch)

===> Epoch[11](1/562): D(x) 0.9359 Loss_D: 0.1114
===> Epoch[11](2/562): D(x) 0.7444 Loss_D: 0.2952
===> Epoch[11](3/562): D(x) 0.9204 Loss_D: 0.0938
===> Epoch[11](4/562): D(x) 0.8743 Loss_D: 0.1726
===> Epoch[11](5/562): D(x) 0.9466 Loss_D: 0.0649
===> Epoch[11](6/562): D(x) 0.8168 Loss_D: 0.1996
===> Epoch[11](7/562): D(x) 0.7909 Loss_D: 0.2263
===> Epoch[11](8/562): D(x) 0.9460 Loss_D: 0.0988
===> Epoch[11](9/562): D(x) 0.9035 Loss_D: 0.1152
===> Epoch[11](10/562): D(x) 0.8322 Loss_D: 0.1906
===> Epoch[11](11/562): D(x) 0.7310 Loss_D: 0.3030
===> Epoch[11](12/562): D(x) 0.9093 Loss_D: 0.1203
===> Epoch[11](13/562): D(x) 0.8394 Loss_D: 0.2062
===> Epoch[11](14/562): D(x) 0.9075 Loss_D: 0.1193
===> Epoch[11](15/562): D(x) 0.9153 Loss_D: 0.1011
===> Epoch[11](16/562): D(x) 0.7508 Loss_D: 0.2606
===> Epoch[11](17/562): D(x) 0.7554 Loss_D: 0.2602
===> Epoch[11](18/562): D(x) 0.8538 Loss_D: 0.1606
===> Epoch[11](19/562): D(x) 0.6822 Loss_D: 0.3358
===> Epoch[11](20/562): D(x) 0.9281 Loss

===> Epoch[11](161/562): D(x) 0.6423 Loss_D: 0.3902
===> Epoch[11](162/562): D(x) 0.8227 Loss_D: 0.2524
===> Epoch[11](163/562): D(x) 0.9320 Loss_D: 0.1110
===> Epoch[11](164/562): D(x) 0.6122 Loss_D: 0.4069
===> Epoch[11](165/562): D(x) 0.9203 Loss_D: 0.1224
===> Epoch[11](166/562): D(x) 0.8217 Loss_D: 0.2058
===> Epoch[11](167/562): D(x) 0.8776 Loss_D: 0.1823
===> Epoch[11](168/562): D(x) 0.8548 Loss_D: 0.2060
===> Epoch[11](169/562): D(x) 0.8206 Loss_D: 0.2475
===> Epoch[11](170/562): D(x) 0.8606 Loss_D: 0.1998
===> Epoch[11](171/562): D(x) 0.7134 Loss_D: 0.3285
===> Epoch[11](172/562): D(x) 0.8812 Loss_D: 0.1808
===> Epoch[11](173/562): D(x) 0.8768 Loss_D: 0.1681
===> Epoch[11](174/562): D(x) 0.8640 Loss_D: 0.2041
===> Epoch[11](175/562): D(x) 0.9631 Loss_D: 0.0843
===> Epoch[11](176/562): D(x) 0.8319 Loss_D: 0.2071
===> Epoch[11](177/562): D(x) 0.8421 Loss_D: 0.2460
===> Epoch[11](178/562): D(x) 0.7686 Loss_D: 0.2903
===> Epoch[11](179/562): D(x) 0.9252 Loss_D: 0.1503
===> Epoch[1

===> Epoch[11](319/562): D(x) 0.9044 Loss_D: 0.1066
===> Epoch[11](320/562): D(x) 0.9126 Loss_D: 0.1035
===> Epoch[11](321/562): D(x) 0.7871 Loss_D: 0.2253
===> Epoch[11](322/562): D(x) 0.7488 Loss_D: 0.2639
===> Epoch[11](323/562): D(x) 0.7102 Loss_D: 0.3027
===> Epoch[11](324/562): D(x) 0.7813 Loss_D: 0.2265
===> Epoch[11](325/562): D(x) 0.7843 Loss_D: 0.2327
===> Epoch[11](326/562): D(x) 0.8670 Loss_D: 0.1483
===> Epoch[11](327/562): D(x) 0.9184 Loss_D: 0.0918
===> Epoch[11](328/562): D(x) 0.7923 Loss_D: 0.2262
===> Epoch[11](329/562): D(x) 0.6801 Loss_D: 0.3345
===> Epoch[11](330/562): D(x) 0.8069 Loss_D: 0.2092
===> Epoch[11](331/562): D(x) 0.8097 Loss_D: 0.2020
===> Epoch[11](332/562): D(x) 0.7336 Loss_D: 0.2744
===> Epoch[11](333/562): D(x) 0.7606 Loss_D: 0.2474
===> Epoch[11](334/562): D(x) 0.8706 Loss_D: 0.1463
===> Epoch[11](335/562): D(x) 0.8299 Loss_D: 0.1780
===> Epoch[11](336/562): D(x) 0.8202 Loss_D: 0.1868
===> Epoch[11](337/562): D(x) 0.7536 Loss_D: 0.2758
===> Epoch[1

===> Epoch[11](477/562): D(x) 0.6990 Loss_D: 0.3138
===> Epoch[11](478/562): D(x) 0.7989 Loss_D: 0.2123
===> Epoch[11](479/562): D(x) 0.7812 Loss_D: 0.2464
===> Epoch[11](480/562): D(x) 0.8989 Loss_D: 0.1122
===> Epoch[11](481/562): D(x) 0.8446 Loss_D: 0.1863
===> Epoch[11](482/562): D(x) 0.9049 Loss_D: 0.1059
===> Epoch[11](483/562): D(x) 0.8770 Loss_D: 0.1296
===> Epoch[11](484/562): D(x) 0.7766 Loss_D: 0.2318
===> Epoch[11](485/562): D(x) 0.9110 Loss_D: 0.1049
===> Epoch[11](486/562): D(x) 0.8252 Loss_D: 0.1865
===> Epoch[11](487/562): D(x) 0.7508 Loss_D: 0.2600
===> Epoch[11](488/562): D(x) 0.8335 Loss_D: 0.1756
===> Epoch[11](489/562): D(x) 0.8384 Loss_D: 0.1740
===> Epoch[11](490/562): D(x) 0.8382 Loss_D: 0.1778
===> Epoch[11](491/562): D(x) 0.6465 Loss_D: 0.3853
===> Epoch[11](492/562): D(x) 0.7030 Loss_D: 0.3076
===> Epoch[11](493/562): D(x) 0.8683 Loss_D: 0.1565
===> Epoch[11](494/562): D(x) 0.8866 Loss_D: 0.1274
===> Epoch[11](495/562): D(x) 0.8542 Loss_D: 0.1555
===> Epoch[1

===> Epoch[12](73/562): D(x) 0.7013 Loss_D: 0.3205
===> Epoch[12](74/562): D(x) 0.8139 Loss_D: 0.1933
===> Epoch[12](75/562): D(x) 0.7764 Loss_D: 0.2391
===> Epoch[12](76/562): D(x) 0.9062 Loss_D: 0.0988
===> Epoch[12](77/562): D(x) 0.8982 Loss_D: 0.1485
===> Epoch[12](78/562): D(x) 0.7723 Loss_D: 0.2502
===> Epoch[12](79/562): D(x) 0.9205 Loss_D: 0.0929
===> Epoch[12](80/562): D(x) 0.7850 Loss_D: 0.2248
===> Epoch[12](81/562): D(x) 0.8735 Loss_D: 0.1607
===> Epoch[12](82/562): D(x) 0.7563 Loss_D: 0.2554
===> Epoch[12](83/562): D(x) 0.6747 Loss_D: 0.3366
===> Epoch[12](84/562): D(x) 0.7295 Loss_D: 0.2751
===> Epoch[12](85/562): D(x) 0.8875 Loss_D: 0.1447
===> Epoch[12](86/562): D(x) 0.8472 Loss_D: 0.1681
===> Epoch[12](87/562): D(x) 0.8852 Loss_D: 0.1320
===> Epoch[12](88/562): D(x) 0.8931 Loss_D: 0.1336
===> Epoch[12](89/562): D(x) 0.7746 Loss_D: 0.2308
===> Epoch[12](90/562): D(x) 0.9187 Loss_D: 0.1064
===> Epoch[12](91/562): D(x) 0.7647 Loss_D: 0.2648
===> Epoch[12](92/562): D(x) 0.

...

===> Epoch[29](329/562): D(x) 0.8258 Loss_D: 0.2127
===> Epoch[29](330/562): D(x) 0.9404 Loss_D: 0.0892
===> Epoch[29](331/562): D(x) 0.9493 Loss_D: 0.0614
===> Epoch[29](332/562): D(x) 0.9892 Loss_D: 0.0201
===> Epoch[29](333/562): D(x) 0.9055 Loss_D: 0.1190
===> Epoch[29](334/562): D(x) 0.8749 Loss_D: 0.1476
===> Epoch[29](335/562): D(x) 0.9440 Loss_D: 0.0662
===> Epoch[29](336/562): D(x) 0.8881 Loss_D: 0.1218
===> Epoch[29](337/562): D(x) 0.9487 Loss_D: 0.0926
===> Epoch[29](338/562): D(x) 0.9258 Loss_D: 0.1049
===> Epoch[29](339/562): D(x) 0.9034 Loss_D: 0.1007
===> Epoch[29](340/562): D(x) 0.9101 Loss_D: 0.1069
===> Epoch[29](341/562): D(x) 0.8535 Loss_D: 0.1737
===> Epoch[29](342/562): D(x) 0.8699 Loss_D: 0.1541
===> Epoch[29](343/562): D(x) 0.9072 Loss_D: 0.1040
===> Epoch[29](344/562): D(x) 0.9793 Loss_D: 0.0336
===> Epoch[29](345/562): D(x) 0.8928 Loss_D: 0.1162
===> Epoch[29](346/562): D(x) 0.9150 Loss_D: 0.0922
===> Epoch[29](347/562): D(x) 0.8432 Loss_D: 0.1829
===> Epoch[2

===> Epoch[29](487/562): D(x) 0.9347 Loss_D: 0.0743
===> Epoch[29](488/562): D(x) 0.8298 Loss_D: 0.1761
===> Epoch[29](489/562): D(x) 0.9036 Loss_D: 0.1059
===> Epoch[29](490/562): D(x) 0.9845 Loss_D: 0.0502
===> Epoch[29](491/562): D(x) 0.9880 Loss_D: 0.0213
===> Epoch[29](492/562): D(x) 0.8658 Loss_D: 0.1389
===> Epoch[29](493/562): D(x) 0.9587 Loss_D: 0.0486
===> Epoch[29](494/562): D(x) 0.9031 Loss_D: 0.1056
===> Epoch[29](495/562): D(x) 0.9448 Loss_D: 0.0722
===> Epoch[29](496/562): D(x) 0.9381 Loss_D: 0.0690
===> Epoch[29](497/562): D(x) 0.8194 Loss_D: 0.2024
===> Epoch[29](498/562): D(x) 0.9515 Loss_D: 0.0800
===> Epoch[29](499/562): D(x) 0.9875 Loss_D: 0.0403
===> Epoch[29](500/562): D(x) 0.7869 Loss_D: 0.2244
===> Epoch[29](501/562): D(x) 0.9272 Loss_D: 0.0835
===> Epoch[29](502/562): D(x) 0.8319 Loss_D: 0.1912
===> Epoch[29](503/562): D(x) 0.9402 Loss_D: 0.0682
===> Epoch[29](504/562): D(x) 0.8610 Loss_D: 0.1538
===> Epoch[29](505/562): D(x) 0.8348 Loss_D: 0.1826
===> Epoch[2

===> Epoch[30](83/562): D(x) 0.8139 Loss_D: 0.1948
===> Epoch[30](84/562): D(x) 0.8661 Loss_D: 0.1374
===> Epoch[30](85/562): D(x) 0.8922 Loss_D: 0.1221
===> Epoch[30](86/562): D(x) 0.8408 Loss_D: 0.1974
===> Epoch[30](87/562): D(x) 0.7779 Loss_D: 0.2476
===> Epoch[30](88/562): D(x) 0.7868 Loss_D: 0.2197
===> Epoch[30](89/562): D(x) 0.8238 Loss_D: 0.2013
===> Epoch[30](90/562): D(x) 0.8814 Loss_D: 0.1461
===> Epoch[30](91/562): D(x) 0.8082 Loss_D: 0.2260
===> Epoch[30](92/562): D(x) 0.7948 Loss_D: 0.2305
===> Epoch[30](93/562): D(x) 0.9174 Loss_D: 0.0889
===> Epoch[30](94/562): D(x) 0.8860 Loss_D: 0.1456
===> Epoch[30](95/562): D(x) 0.9291 Loss_D: 0.1202
===> Epoch[30](96/562): D(x) 0.8867 Loss_D: 0.1289
===> Epoch[30](97/562): D(x) 0.9556 Loss_D: 0.0669
===> Epoch[30](98/562): D(x) 0.9194 Loss_D: 0.0918
===> Epoch[30](99/562): D(x) 0.8332 Loss_D: 0.1784
===> Epoch[30](100/562): D(x) 0.8107 Loss_D: 0.2027
===> Epoch[30](101/562): D(x) 0.8094 Loss_D: 0.2065
===> Epoch[30](102/562): D(x)

===> Epoch[30](241/562): D(x) 0.9379 Loss_D: 0.0707
===> Epoch[30](242/562): D(x) 0.8799 Loss_D: 0.1564
===> Epoch[30](243/562): D(x) 0.8359 Loss_D: 0.1720
===> Epoch[30](244/562): D(x) 0.8847 Loss_D: 0.1410
===> Epoch[30](245/562): D(x) 0.8814 Loss_D: 0.1408
===> Epoch[30](246/562): D(x) 0.9554 Loss_D: 0.1117
===> Epoch[30](247/562): D(x) 0.7588 Loss_D: 0.2508
===> Epoch[30](248/562): D(x) 0.7364 Loss_D: 0.2750
===> Epoch[30](249/562): D(x) 0.8338 Loss_D: 0.1723
===> Epoch[30](250/562): D(x) 0.8148 Loss_D: 0.1917
===> Epoch[30](251/562): D(x) 0.8044 Loss_D: 0.2141
===> Epoch[30](252/562): D(x) 0.7621 Loss_D: 0.2490
===> Epoch[30](253/562): D(x) 0.8889 Loss_D: 0.1389
===> Epoch[30](254/562): D(x) 0.7542 Loss_D: 0.2620
===> Epoch[30](255/562): D(x) 0.9431 Loss_D: 0.0998
===> Epoch[30](256/562): D(x) 0.7035 Loss_D: 0.3068
===> Epoch[30](257/562): D(x) 0.8559 Loss_D: 0.1637
===> Epoch[30](258/562): D(x) 0.8704 Loss_D: 0.1469
===> Epoch[30](259/562): D(x) 0.8504 Loss_D: 0.1612
===> Epoch[3

===> Epoch[30](399/562): D(x) 0.7980 Loss_D: 0.2100
===> Epoch[30](400/562): D(x) 0.9318 Loss_D: 0.0844
===> Epoch[30](401/562): D(x) 0.8161 Loss_D: 0.2136
===> Epoch[30](402/562): D(x) 0.8954 Loss_D: 0.1203
===> Epoch[30](403/562): D(x) 0.8805 Loss_D: 0.1564
===> Epoch[30](404/562): D(x) 0.8558 Loss_D: 0.1534
===> Epoch[30](405/562): D(x) 0.7250 Loss_D: 0.2943
===> Epoch[30](406/562): D(x) 0.7535 Loss_D: 0.2696
===> Epoch[30](407/562): D(x) 0.7810 Loss_D: 0.2289
===> Epoch[30](408/562): D(x) 0.7699 Loss_D: 0.2761
===> Epoch[30](409/562): D(x) 0.8393 Loss_D: 0.1979
===> Epoch[30](410/562): D(x) 0.9086 Loss_D: 0.1194
===> Epoch[30](411/562): D(x) 0.9048 Loss_D: 0.1141
===> Epoch[30](412/562): D(x) 0.8322 Loss_D: 0.1743
===> Epoch[30](413/562): D(x) 0.8279 Loss_D: 0.1899
===> Epoch[30](414/562): D(x) 0.7340 Loss_D: 0.2890
===> Epoch[30](415/562): D(x) 0.8472 Loss_D: 0.2067
===> Epoch[30](416/562): D(x) 0.7372 Loss_D: 0.2859
===> Epoch[30](417/562): D(x) 0.8642 Loss_D: 0.1839
===> Epoch[3